<span style="font-size:50px"><strong>ML4HC Project 1 - Task 4</strong></span>


In [21]:
import os
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm
from matplotlib import pyplot as plt

# Define paths
data_dir = "../ml4h_data/p1/"#"data"

pd.set_option('display.max_columns', None)


#  Prompting an LLM to solve a time-series problem - Q4.1

In [ ]:
import ollama

# Example of running a model in Ollama with a properly formatted message
response = ollama.chat(
    "llama3.2", 
    messages=[{"role": "user", "content": "Hello, Ollama! How are you?"}]
)

# Output the response
print(response['message']['content'])

In [45]:
import pandas as pd
import ollama
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, average_precision_score
from ollama import Client

# Initialize Ollama client
client = Client()

def create_patient_summaries(df):
    summaries = {}
    grouped = df.groupby("RecordID")
    for rid, group in grouped:
        features = []
        for var in group["Variable"].unique():
            vals = group[group["Variable"] == var]["Value"]
            if not vals.empty:
                stats = f"{var}: min={vals.min():.2f}, max={vals.max():.2f}, mean={vals.mean():.2f}"
                features.append(stats)
        text = f"Patient {rid} summary: " + ", ".join(features)
        summaries[rid] = text
    return summaries

# LLM prompt with chain-of-thought and prediction at the end
def query_llm(prompt):
    response = client.chat(
        model="llama3.2",
        messages=[
            {
                "role": "user",
                "content": (
                    prompt +
                    "\nBased on the above medical features, reason step-by-step using medical knowledge "
                    "to determine whether the patient will die in hospital but never print your reasoning . The only thing you should return is only one word: Yes or No."
                )
            }
        ]
    )
    return response["message"]["content"].strip()

def predict_with_llm(summaries, max_patients=None):
    predictions = {}
    for i, (rid, text) in enumerate(tqdm(summaries.items())):
        if max_patients and i >= max_patients:
            break
        full_response = query_llm(text)
        final_word = full_response.strip().split()[-1].lower()

        if final_word == "yes" or final_word == "yes.":
            pred = 1
        elif final_word == "no" or final_word == "no.":
            pred = 0
        else:
            pred = None

        predictions[rid] = pred
        print(f"\nRecordID: {rid}")
        print(full_response)
        print(f"→ Prediction: {pred}\n")
    return predictions

def evaluate_predictions(preds, y_true):
    valid = {rid: pred for rid, pred in preds.items() if pred is not None}
    y_pred = [valid[rid] for rid in y_true.index if rid in valid]
    y_true_vals = [y_true[rid] for rid in y_true.index if rid in valid]

    auroc = roc_auc_score(y_true_vals, y_pred)
    auprc = average_precision_score(y_true_vals, y_pred)
    print(f"LLM AUROC: {auroc:.4f}, AUPRC: {auprc:.4f}")


In [46]:
df = pd.read_parquet("data_c_scaled_nonImputed.parquet")
df = df.melt(id_vars=["RecordID", "Time"], var_name="Variable", value_name="Value")

# Find RecordIDs where in-hospital death == 1
positive_ids = df[(df["Variable"] == "In-hospital_death") & (df["Value"] == 1)]["RecordID"].unique()

# Filter df to only those records
df = df[df["RecordID"].isin(positive_ids)].reset_index(drop=True)


y_true = df[df["Variable"] == "In-hospital_death"].groupby("RecordID")["Value"].first()
 

In [47]:
predssummaries = create_patient_summaries(df)
preds = predict_with_llm(summaries, max_patients=5)
evaluate_predictions(preds, y_true)



  0%|          | 1/585 [00:04<44:01,  4.52s/it]


RecordID: 152873.0
No.
→ Prediction: 0



  0%|          | 2/585 [00:06<31:01,  3.19s/it]


RecordID: 152893.0
No.
→ Prediction: 0



  1%|          | 3/585 [00:08<26:26,  2.73s/it]

...

  1%|          | 4/585 [00:10<23:35,  2.44s/it]


RecordID: 152938.0
No.
→ Prediction: 0



  1%|          | 5/585 [00:13<25:28,  2.63s/it]


RecordID: 152956.0
No
→ Prediction: 0

LLM AUROC: nan, AUPRC: 1.0000



/opt/homebrew/anaconda3/envs/ml4hc/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


#  Using LLMs to retrieve embeddings - Q4.2

#  Using time-series foundation models - Q4.3